In [1]:
import numpy as np
from specutils.analysis import equivalent_width
from specutils.analysis import line_flux
from specutils import Spectrum1D
from specutils import SpectralRegion
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import astropy.units as u

In [2]:
SDSS_EMLINES = {
    'OII_3726': {'cen':3726.032, 'low':3717.0, 'upp':3737.0},
    'OII_3729': {'cen':3728.815, 'low':3717.0, 'upp':3737.0},
    'NeIII_3869': {'cen':3869.060, 'low':3859.0, 'upp':3879.0},
    'H_delta': {'cen':4101.734, 'low':4092.0, 'upp':4111.0},
    'H_gamma': {'cen':4340.464, 'low':4330.0, 'upp':4350.0},
    'OIII_4363': {'cen':4363.210, 'low':4350.0, 'upp':4378.0},
    'H_beta': {'cen':4861.325, 'low':4851.0, 'upp':4871.0},
    'OIII_4959': {'cen':4958.911, 'low':4949.0, 'upp':4969.0},
    'OIII_5007': {'cen':5006.843, 'low':4997.0, 'upp':5017.0},
    'HeI_5876': {'cen':5875.67, 'low':5866.0, 'upp':5886.0},
    'OI_6300': {'cen':6300.304, 'low':6290.0, 'upp':6310.0},
    'NII_6548': {'cen':6548.040, 'low':6533.0, 'upp':6553.0},
    'H_alpha': {'cen':6562.800, 'low':6553.0, 'upp':6573.0},
    'NII_6584': {'cen':6583.460, 'low':6573.0, 'upp':6593.0},
    'SII_6717': {'cen':6716.440, 'low':6704.0, 'upp':6724.0},
    'SII_6731': {'cen':6730.810, 'low':6724.0, 'upp':6744.0},
    'ArIII7135': {'cen':7135.8, 'low':7130.0, 'upp':7140.0}
}

In [3]:
def measure_flux(wspec, spec, emline,wave_margin=300,redshift=0.0323,cont=0):
    
    wave_flag = ((wspec >= emline['cen']*(1.0 + redshift) - wave_margin) &
                 (wspec <= emline['cen']*(1.0 + redshift) + wave_margin))

    wave_use = wspec[wave_flag]
    spec_use = spec[wave_flag]

    flux_line = line_flux(        
        Spectrum1D(
            spectral_axis=wave_use * u.AA,
            flux = (spec_use-cont) * u.Unit('erg cm-2 s-1 AA-1')
        ),
        regions=SpectralRegion(emline['low'] * u.AA * (1.0 + redshift),
                               emline['upp'] * u.AA * (1.0 + redshift)),
    ).value

    return flux_line

def correct_dust(x,flux):
    k = 2.656*(-2.156+1.509/x-0.198/x**2+0.011/x**3)+4.88
    flux_corr = flux/10**(-0.4*k*EBV)
    return flux_corr

In [4]:
spectra_10k=np.genfromtxt('spec1d.mn46.005.lid_2623.txt',names=('LAMBDA','IVAR','FLUX'))
wave_10k = spectra_10k['LAMBDA']
spec_10k = spectra_10k['FLUX']

spectra = fits.open('bluedot_coadd_det3.fits')[1].data
wave = spectra['wave']
spec = spectra['flux']

In [5]:
Ha_10k = measure_flux(wave_10k, spec_10k, SDSS_EMLINES['H_alpha'],redshift=0.0323,cont=300)
Hbeta_10k = measure_flux(wave_10k, spec_10k, SDSS_EMLINES['H_beta'],redshift=0.0323,cont=300)
OIII4959_10k = measure_flux(wave_10k, spec_10k, SDSS_EMLINES['OIII_4959'],redshift=0.0323,cont=300)
OIII5007_10k = measure_flux(wave_10k, spec_10k, SDSS_EMLINES['OIII_5007'],redshift=0.0323,cont=300)
NII6584_10k = measure_flux(wave_10k, spec_10k, SDSS_EMLINES['NII_6584'],redshift=0.0323,cont=300)
SII6717_10k = measure_flux(wave_10k, spec_10k, SDSS_EMLINES['SII_6717'],redshift=0.0323,cont=300)
SII6731_10k = measure_flux(wave_10k, spec_10k, SDSS_EMLINES['SII_6731'],redshift=0.0323,cont=300)
EBV = 0.934*np.log10((Ha_10k/Hbeta_10k)/2.86)

In [6]:
Ha_10k_corr = correct_dust(0.65628,Ha_10k)
Hbeta_10k_corr = correct_dust(0.48613,Hbeta_10k)
OIII4959_10k_corr = correct_dust(0.49589,OIII4959_10k)
OIII5007_10k_corr = correct_dust(0.5007,OIII5007_10k)
NII6584_10k_corr = correct_dust(0.6584,NII6584_10k)
SII6717_10k_corr = correct_dust(0.6717,SII6717_10k)
SII6731_10k_corr = correct_dust(0.6731,SII6731_10k)

In [7]:
O3N2 = np.log10(OIII5007_10k_corr/Hbeta_10k_corr*Ha_10k_corr/NII6584_10k_corr)
metallicty_O3N2 = 8.505 - 0.221 * O3N2
N2 = np.log10(NII6584_10k_corr/Ha_10k_corr)
metallicty_N2 = 8.667 + 0.455 * N2
N2S2 = np.log10(NII6584_10k_corr/(SII6717_10k_corr+SII6731_10k_corr))
metallicty_N2S2 = 8.77 + N2S2 + 0.264 * N2
print(metallicty_O3N2,metallicty_N2,metallicty_N2S2)

8.037756412420965 8.035641669856036 7.870084439862348


In [8]:
Ha = measure_flux(wave, spec, SDSS_EMLINES['H_alpha'],redshift=0.0323,cont=100)
Hbeta = measure_flux(wave, spec, SDSS_EMLINES['H_beta'],redshift=0.0323,cont=100)
OIII4959 = measure_flux(wave, spec, SDSS_EMLINES['OIII_4959'],redshift=0.0323,cont=100)
OIII5007 = measure_flux(wave, spec, SDSS_EMLINES['OIII_5007'],redshift=0.0323,cont=100)
NII6584 = measure_flux(wave, spec, SDSS_EMLINES['NII_6584'],redshift=0.0323,cont=100)
SII6717 = measure_flux(wave, spec, SDSS_EMLINES['SII_6717'],redshift=0.0323,cont=100)
SII6731 = measure_flux(wave, spec, SDSS_EMLINES['SII_6731'],redshift=0.0323,cont=100)
EBV = 0.934*np.log10((Ha/Hbeta)/2.86)

In [9]:
Ha_corr = correct_dust(0.65628,Ha)
Hbeta_corr = correct_dust(0.48613,Hbeta)
OIII4959_corr = correct_dust(0.49589,OIII4959)
OIII5007_corr = correct_dust(0.5007,OIII5007)
NII6584_corr = correct_dust(0.6584,NII6584)
SII6717_corr = correct_dust(0.6717,SII6717)
SII6731_corr = correct_dust(0.6731,SII6731)

In [10]:
O3N2 = np.log10(OIII5007_corr/Hbeta_corr*Ha_corr/NII6584_corr)
metallicty_O3N2 = 8.505 - 0.221 * O3N2
N2 = np.log10(NII6584_corr/Ha_corr)
metallicty_N2 = 8.667 + 0.455 * N2
N2S2 = np.log10(NII6584_corr/(SII6717_corr+SII6731_corr))
metallicty_N2S2 = 8.77 + N2S2 + 0.264 * N2
print(metallicty_O3N2,metallicty_N2,metallicty_N2S2)

8.069618086437565 8.183798753172784 7.908347185448563
